TO DO:
- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries
- Include Active/Inactive column in CSV for active/inactive coins
- Update_CSV_to_Latest should contain active coins from Exchanges and From CSV. Check if a (coin,exchange) tuple is active(Check if it is present on exchange using ccxt library) . If it is active   get latest data for it if already present in CSV,if not in CSV get all data. If coin is not active on exchange , we will put a Active/Inactive status in CSV accordingly. All functions will have   to be modified to run code only for active coin-exchange combinations.
- Date Format will be '%d-%m-%Y %H:%M:%S' . This is giving me a lot of problems especially while reading data. When I don't put :%S it tells me dataframe has second and sometimes when :%S is         there, it tells me no second value in dataframe.
- Analytics Value Accuracy. Some parameter in Jupyter.
- Have to fetch Coins based on Parameters. Example - Fetch all active coins-exchange combinations where RSI>0 and RSI<=30. Fetch all active coins-exchange combinations where closing price is         between LOWERBAND and MIDDLEBAND. Get me intersection(common coins) of these 2 list. Now the coin from the intersection list which probably has the lowest volume can increase in price faster       then the others(Little increase in Volume will result in Big increase in Price)
- For each active coin-exchange combination I want to check the change in Value of different Technical Indicators of 2 consecutive periods in time. Example - I want to know if for a particular       coin RSI=a on period x and RSI>a on period x+1. I want to know whenever MACD and MACD_SIGNAL cross each other(On period x MACD=a and MACD_SIGNAL=b where a<=b and on period x+1 MACD=a and           MACD_SIGNAL=b where a>b. MACD_HISTOGRAM same like RSI want to know when it is 'a' on period x and 'a++' on period x+1).
- Convert 1D timeframe to 3D/1Week/etc. Convert 1H timeframe to 4H/6H/etc.
- Batch Processing of downloading new coins data

In [1]:
# Imports
import os
import time
import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import ccxt
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
from apscheduler.schedulers.background import BackgroundScheduler

In [2]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.
                    INFO)


In [3]:
cryptocompare_wrapper = os.path.join(os.curdir, 'cryptocompare_wrapper.py')
cryptocompare_wrapper

'.\\cryptocompare_wrapper.py'

In [4]:
import cryptocompare_wrapper as ccw
reload(ccw)

<module 'cryptocompare_wrapper' from 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\notebooks\\cryptocompare_wrapper.py'>

In [5]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'
datetimeStringformat_to_csv = "%d-%m-%Y %H:%M"
data_directory = 'D:\\crypto_analysis-master-master\\crypto_analysis-master-master\\data\\'

In [6]:
# Defaults
CURR = 'BTC'
EXCHANGE = 'CCCAGG'
COIN = 'ETH'
COIN_LIST = ['BTC', 'ETH', 'XRP']
EXCHANGES = ['Bittrex','Binance','Kucoin','HuobiPro','Cryptopia','IDEX']
EXCHANGES = ['Binance']
bittrex_exchange = ccxt.bittrex()
binance_exchange = ccxt.binance()
kucoin_exchange = ccxt.kucoin()
huobiPro_exchange = ccxt.huobipro()
cryptopia_exchange = ccxt.cryptopia()
bitmex_exchange = ccxt.bitmex()
#print(bittrex_exchange.fetchCurrencies())
list_of_exchanges = [bittrex_exchange,binance_exchange,kucoin_exchange,huobiPro_exchange,
                     cryptopia_exchange]
#list_of_exchanges = [bitmex_exchange]

In [7]:
# For every exchange, fetch it's markets. Then depending on the JSON returned, prepare a list of coins for which historical data has to be downloaded.
def setupExchanges(list_of_exchanges):
    done = False
    i=0
    #df_markets = pd.DataFrame(markets)
    #bittrex_market = bittrex_exchange.fetchMarkets()
    #binance_market = binance_exchange.fetchMarkets()
    #kucoin_market = kucoin_exchange.fetchMarkets()
    #list_of_markets = [#bittrex_market,
                       #binance_market
     #                  kucoin_market #For kucoin the fetchMarkets function returns different dictionary keys
      #                  ]

    var_quote = ""
    coin_exchange_combination = {}
    for exchange in list_of_exchanges:
        coins_list = set()
        #if exchange.name == 'Cryptopia' or exchange.name == 'Bittrex' or exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
            #continue #exchange.name == 'Binance' or 
        markets = exchange.fetchMarkets()
        for row in markets:
            if exchange.name == 'Huobi Pro' or exchange.name == 'Cryptopia':
                if row['base'] not in coins_list:
                        coins_list.add(row['base'])
                #continue
            if  'active' in row and row['active'] == True :
                #print(exchange.name,row)
                #sys.exit("Te")
                if exchange.name == 'Bittrex' or exchange.name == 'Binance'  :
                    var_quote = "quoteId"
                elif   exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
                    var_quote = "quote"
                #print(var_quote)
                if var_quote in row and row[var_quote] == 'BTC':
                    if row['base'] not in coins_list:
                        coins_list.add(row['base'])
        coin_exchange_combination[exchange.name] = coins_list
    return coin_exchange_combination

In [8]:
def update_and_delete_coin_exchange_combination(csv_filename_read,csv_filename_write,timeframe,coin_exchange_combination):
    #For every exchange, download the new coins and delete coins which are delisted from that exchange
    csv_all_coins_full = csv_filename_read
    csv_all_coins_full_new = csv_filename_write
    not_updated = defaultdict(list)
    existing_coin_exchange = []
    # If the csv already exists, find out which coins and exchanges have already been added
    if os.path.isfile(csv_all_coins_full):
        df_csv_all_coins_full = pd.read_csv(csv_all_coins_full,index_col=['exchange','coin'])
        # existing_coin_exchange is a list of tuples (coin, exchange)
        existing_coin_exchange = np.unique(df_csv_all_coins_full.index.values)
        coin_exchange_combination_in_excel = {}
        for a, b in existing_coin_exchange:
            coin_exchange_combination_in_excel.setdefault(a, []).append(b)
    number_of_coins = 0
    coin_exchange_combination_to_delete = {}
    for exchange in coin_exchange_combination :
        if exchange not in coin_exchange_combination_in_excel:
            continue
        coins_list_from_exchange = coin_exchange_combination[exchange]
        coins_list_from_excel = coin_exchange_combination_in_excel[exchange]
        coins_to_download = list(set(coins_list_from_exchange) - set(coins_list_from_excel))
        coins_to_delete = list(set(coins_list_from_excel) - set(coins_list_from_exchange))
        coin_exchange_combination_to_delete[exchange] = coins_to_delete
        print(exchange,coins_to_download)
        for symbol in coins_to_download:
         #For every symbol-exchange combination, if it is present in CSV,don't download historical Data for it.
            try:
                # Can't fetch the same symbol in same symbol rate
                print(symbol,exchange,"In Try Block")
                func = function_period_mapping[timeframe]
                to_curr = 'BTC'
                if symbol == "BTC":
                    to_curr = "USD"
                if exchange == 'IDEX':
                    to_curr = 'ETH'
                if symbol is not to_curr:
                    df_coin_all = func(
                        coin=symbol,
                        to_curr=to_curr,
                        timestamp=time.time(),
                        exchange=exchange
                    )
    
                if df_coin_all.empty:
                    not_updated[exchange].append(symbol)
                    #print(symbol,exchange)
                else:
                    df_coin_all['exchange'] = exchange
                    df_coin_all['coin'] = symbol
                    #print("Coin Inserted")
                    df_coin_all['time'] = pd.to_datetime(df_coin_all.unix_timestamp,unit="s",utc=True)
                    df_coin_all = df_coin_all.reset_index().set_index(['coin','exchange', 'unix_timestamp'])
                    #print("Index Set Again")
                    # If csv does not exist, write, else append
                    if not os.path.isfile(csv_all_coins_full_new):
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='w')
                    else:
                        df_coin_all.to_csv(csv_all_coins_full_new, mode='a', header=False)
                    number_of_coins = number_of_coins +1
    
            except Exception as e:
                logging.error(e)
                # logging.debug("Could not update data for {curr} from {exchange}".format(curr=symbol, exchange=exchange))
                not_updated[exchange].append(symbol)
    
    logging.error("Did not update the following. Try again.\n {not_updated}".format(not_updated=not_updated))
    print(number_of_coins)
    print(coin_exchange_combination_to_delete)
    delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full.reset_index(),csv_filename_read)

In [9]:
def delete_coins_from_CSV(coin_exchange_combination_to_delete,df_csv_all_coins_full,csv_filename_read):
    for exchange in coin_exchange_combination_to_delete:
        coins_list_to_delete = coin_exchange_combination_to_delete[exchange]
        for symbol in coins_list_to_delete:
            df_csv_all_coins_full = df_csv_all_coins_full[~((df_csv_all_coins_full['coin'] == symbol) & (df_csv_all_coins_full['exchange'] == exchange))] 
    df_csv_all_coins_full.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename_read)

In [ ]:
coins_list_from_exchange = setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_day_full_1day.csv','D:\crypto_analysis-master-master\crypto_analysis-master-master\data\all_coins_day_full_1day_new_coins.csv','1dayfull',coins_list_from_exchange)

In [ ]:
#coins_list = ["BTC"]
coins_list = [""]
setupExchanges(list_of_exchanges)
update_and_delete_coin_exchange_combination(data_directory+'all_coins_hour_full_1hour_.csv','D:\crypto_analysis-master-master\crypto_analysis-master-master\data\all_coins_hour_full_1hour_.csv','1hour',coins_list)

In [10]:
# Maps csv (future data objects) to period granularity
# If we store all data together in a single data source, we'll change this to a function which returns corresponding rows
data_csv_period_mapping = {
    "1day": data_directory+'all_coins_day_full_1day.csv',
    "1hour": data_directory+'all_coins_hour_full_1hour.csv',
    "1min": data_directory+'all_coins_min_full_1min.csv',
    "1daycryptopia":data_directory+'all_coins_day_full_1day_Cryptopia.csv',
    '1daybtcbitfinex':data_directory+'BTC_Bitfinex_day_full_1day.csv',
    '1hourbtcbitfinex':data_directory+'BTC_Bitfinex_hour_full_1hour.csv'
}
frequency_resampling_period_mapping = {
    "day":'D',
    "hour":'H',
    "min":'M'
}
function_period_mapping = {
    '1day': ccw.get_historical_price_day,
    '1hour': ccw.get_historical_price_hour,
    '1min': ccw.get_historical_price_minute,
    '1dayfull' : ccw.get_historical_price_day_full,
    '1daycryptopia' : ccw.get_historical_price_day,
    '1daybtcbitfinex':ccw.get_historical_price_day,
    '1hourbtcbitfinex':ccw.get_historical_price_hour
}

In [12]:
#df_csv = pd.read_csv('all_coins_day_full.csv', index_col=None)
indicator_list = ['unix_timestamp','BBANDS_BANDWIDTH_PERCENT','MONEY_FLOW_INDEX',
                   'STOCH_PERCENT_K_MONEY_FLOW_INDEX','STOCH_PERCENT_D_MONEY_FLOW_INDEX','RSI','RSI_OVER_BOUGHT','RSI_OVER_SOLD',
                   'STOCHRSI_K','STOCHRSI_D','STOCH_PERCENT_K','STOCH_PERCENT_D','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST',
                  'MACD','MACD_SIGNAL','MACD_TEST','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST']   

In [14]:
 # Technical Analysis Settings
EMA_FAST = 10
EMA_SLOW = 20
RSI_PERIOD = 14
RSI_OVER_BOUGHT = 70
RSI_OVER_SOLD = 30
RSI_AVG_PERIOD = 15
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
STOCH_K = 14
STOCH_D = 3
STOCH_OVER_BOUGHT = 70
STOCH_OVER_SOLD = 30
from pyti import bollinger_bands
from pyti import money_flow_index
from pyti import stochastic
from pyti import simple_moving_average
from pyti import stochrsi
from pyti import on_balance_volume

In [13]:
def update_indicator(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    df_csv.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['BBANDS_BANDWIDTH_PERCENT'] = pyti.bollinger_bands.percent_b(req_data2.close.values,20)
            req_data2['MONEY_FLOW_INDEX'] = money_flow_index.money_flow_index(req_data2.close.values, req_data2.high.values, req_data2.low.values, np_volumeto, 14)
            req_data2['STOCH_PERCENT_K_MONEY_FLOW_INDEX'] = pyti.stochastic.percent_k(req_data2.MONEY_FLOW_INDEX.values,14) * 100
            req_data2['STOCH_PERCENT_D_MONEY_FLOW_INDEX'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K_MONEY_FLOW_INDEX.values,3)
            req_data2['RSI'] = talib.RSI(req_data2.close.values, timeperiod=RSI_PERIOD)
            req_data2['RSI_OVER_BOUGHT'] = np.where((req_data2.RSI >= RSI_OVER_BOUGHT) & (req_data2.RSI <= req_data2.RSI.shift(1)),1,0)
            req_data2['RSI_OVER_SOLD'] = np.where((req_data2.RSI <= RSI_OVER_SOLD) & (req_data2.RSI >= req_data2.RSI.shift(1)),1,0)
            req_data2['STOCHRSI_K'] = pyti.stochrsi.stochrsi(req_data2.close.values,14)
            req_data2['STOCHRSI_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCHRSI_K.values,3)
            req_data2['STOCH_PERCENT_K'] = pyti.stochastic.percent_k(req_data2.high.values,14) * 100
            req_data2['STOCH_PERCENT_D'] = pyti.simple_moving_average.simple_moving_average(req_data2.STOCH_PERCENT_K.values,3)
            req_data2['STOCH_OVER_BOUGHT'] = np.where((req_data2.STOCH_PERCENT_K >= STOCH_OVER_BOUGHT) & (req_data2.STOCH_PERCENT_K <= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['STOCH_OVER_SOLD'] = np.where((req_data2.STOCH_PERCENT_K <= STOCH_OVER_SOLD) & (req_data2.STOCH_PERCENT_K >= req_data2.STOCH_PERCENT_K.shift(1)),1,0)
            req_data2['SMA_FAST'] = talib.SMA(req_data2.close.values,7)
            req_data2['SMA_SLOW'] = talib.SMA(req_data2.close.values,21)
            req_data2['SMA_TEST'] = np.where(req_data2.SMA_FAST>req_data2.SMA_SLOW,1,0)
            req_data2['ON_BALANCE_VOLUME'] = on_balance_volume.on_balance_volume(req_data2.close.values,np_volumeto)
            req_data2['ON_BALANCE_VOLUME_TEST'] = np.where(req_data2.ON_BALANCE_VOLUME>req_data2.ON_BALANCE_VOLUME.shift(1),1,0)
            """
            req_data2['Accumulation_Distribution_Oscillator'] = talib.ADOSC(req_data2.high.values,req_data2.low.values
                                                  ,req_data2.close.values,np_volumeto)
            req_data2['ADOSC_TEST'] = np.where((req_data2.Accumulation_Distribution_Oscillator>req_data2.Accumulation_Distribution_Oscillator.shift(1)) & (req_data2.Accumulation_Distribution_Oscillator>=0) & 
                                               (req_data2.Accumulation_Distribution_Oscillator.shift(1)<=0),1,0)
            """
            
            req_data2['MACD'],req_data2['MACD_SIGNAL'],MACD_HISTOGRAM= talib.MACD(req_data2.close.values,fastperiod=MACD_FAST,slowperiod=MACD_SLOW,signalperiod=MACD_SIGNAL)
            req_data2['MACD_TEST'] = np.where(req_data2.MACD>req_data2.MACD_SIGNAL,1,0)
            
            
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_10days.csv',250,'10day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_1day_Cryptopia.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator(data_directory+'all_coins_day_full_3days_Cryptopia.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_7days_Cryptopia.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_day_full_14days_Cryptopia.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator(data_directory+'all_coins_min_full_1min.csv',250,'1min')
print("Done")

In [15]:
def resample(csv_filename,period,resampling_multiplier,exchange,datetimeformat_string,output_file_name):
    df_csv = pd.read_csv(csv_filename,dayfirst=True)
    #df_csv['time'] = df_csv.unix_timestamp.apply(lambda t: datetime.datetime.fromtimestamp(int(t)).strftime(datetimeformat_string))
    df_csv.unix_timestamp = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    #df_csv['time'].to_csv('time.csv')
    df_csv = df_csv.reset_index()
    #print(df_csv.columns)
    for indicator in indicator_list:
        if indicator not in df_csv.columns:
            df_csv[indicator] = np.nan
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    all_dataframes = []
    resampling_period = ""+str(resampling_multiplier)+frequency_resampling_period_mapping[period]
    output_csv_filename = output_file_name
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            #print(type(req_data.index))
            req_data = req_data.resample(resampling_period,level=2,closed='right',label='right').agg({'open': 'first', 
                                     'high': 'max', 
                                     'low': 'min', 
                                     'close': 'last',
                                    'volumeto':'sum',
                                        'volumefrom':'sum'})

            req_data['coin'] = coin_name
            req_data['exchange'] = key
            
            #print(req_data)
            req_data = req_data.reset_index()
            req_data['unix_timestamp'] =  req_data.unix_timestamp.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t), '%Y-%m-%d %H:%M:%S').timetuple()))
            #req_data['time'].to_csv("Req_data_Time.csv")
            req_data['time'] = pd.to_datetime(req_data.unix_timestamp,unit="s",utc=True)

            req_data = req_data.set_index(['coin','exchange','unix_timestamp'])
            i = i+1
            print(coin_name,i)
            #req_data = req_data.drop(labels=['time'],axis=1)
            all_dataframes.append(req_data)
    dataframe_answer = pd.concat(all_dataframes)
    #dataframe_answer['time'].to_csv('Time.csv')
    dataframe_answer.to_csv(output_csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
resample(data_csv_period_mapping['1day'],'day',3,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days.csv')

In [ ]:
resample(data_csv_period_mapping['1day'],'day',7,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days.csv')

In [ ]:
resample(data_csv_period_mapping['1day'],'day',10,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_10days.csv')

In [ ]:
resample(data_csv_period_mapping['1day'],'day',14,"",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',3,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_3days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',7,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_7days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1daycryptopia'],'day',14,"Cryptopia",datetimeStringformat_to_csv,data_directory+'all_coins_day_full_14days_Cryptopia.csv')

In [ ]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',4,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_4hours.csv')

In [ ]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',6,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_6hours.csv')

In [ ]:
resample(data_csv_period_mapping['1hourbtcbitfinex'],'hour',12,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_hour_full_12hours.csv')

In [ ]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',3,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_3days.csv')

In [ ]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',7,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_7days.csv')

In [ ]:
resample(data_csv_period_mapping['1daybtcbitfinex'],'day',14,"Bitfinex",datetimeStringformat_to_csv,data_directory+'BTC_Bitfinex_day_full_14days.csv')

In [16]:
def fetch_data_api(coin=COIN, to_curr=CURR, nperiods=1, period='1day',exchange_name=EXCHANGES[0]):
    """ Fetch data for coin over nperiods
        e.g. Get data for 'BTC' for past 12 hours in hours granularity
    """
    period = period.lower()
    func = function_period_mapping[period]
    if exchange_name == 'IDEX':
        to_curr = 'ETH'
    if coin == 'BTC':
        to_curr = 'USD'
    coin_last_nperiods = func(
        coin=coin,
        to_curr=to_curr,
        limit=nperiods,
        exchange=exchange_name
    )
    #print(coin_last_nperiods.index.get_level_values(0))
    #answer = coin_last_nperiods.iloc[-int(nperiods):]
    #print(answer.index.get_level_values(0))
    if coin_last_nperiods is not None:
        return coin_last_nperiods.iloc[-int(nperiods):]
    else:
        return None

In [17]:
def delete_latest_period_data(csv_filename,datetimeformat_string):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    final_dataframe = []
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data = req_data[:-1]
            #print(req_data.tail(1))
            final_dataframe.append(req_data)
    answer = pd.concat(final_dataframe).reset_index()
    #answer.to_csv('answer.csv')
    return answer

In [18]:
def update_csv_to_latest(period='1day',datetimeformat_string='%d-%m-%Y %H:%M',final_csv_column_order=[]):
    """ Update the csv for given period upto current time for coin """
    period = period.lower()
    csv_filename = data_csv_period_mapping[period]  # Get corr csv
    #csv_filename = 'Experiment.csv'
    #df_coin_period = pd.read_csv(csv_filename)  # , index_col=['coin', 'exchange']
    df_coin_period = delete_latest_period_data(csv_filename,datetimeStringformat_to_csv)
    #print(df_coin_period.tail(1)['time'])
    print("Updated Dataframe")
    csv_column_order = df_coin_period.columns.tolist()
    df_coin_period = df_coin_period.set_index(keys=['coin', 'exchange'])
    #df_coin_period.time = df_coin_period.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    
    lst_new_data = []
    PRINT_MSG = "{:15} {!s:20} {!s:>20} {:>10}"
    logging.info(PRINT_MSG.format("Exchange", "Last Updated Time", "Elapsed Time", "nPeriodsAgo"))
    coins_in_csv = list(df_coin_period.index.get_level_values(0).unique())
    #coins_in_csv = ['ZRX']
    i=0
    j=0
    for coin in coins_in_csv:
        df_coin_period_coin = df_coin_period.loc[coin]
        # Group by exchange, sort on timestamp, and get the last row of that particular coin
        last_update = df_coin_period_coin.groupby('exchange', group_keys=False).apply(lambda c: c.sort_values(by='unix_timestamp').tail(1))
        logging.info("-" * 10 + " For coin - {}".format(coin))
        
        for exchange in last_update.index.values: #For every coin exchange combination
            last_updated_time = int(last_update.loc[exchange]['unix_timestamp']) #Get the time of the last row
            #print(type(last_updated_time))
            try:
                # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%Y-%m-%d %H:%M:%S')
                # elapsed_time = datetime.datetime.now() - datetime.datetime.strptime(last_updated_time, '%d-%m-%Y %H:%M')
                elapsed_time = int(time.time()) - last_updated_time
            except ValueError as e:
                logging.info("Failed to parse time {} for {}--{}".format(last_updated_time, coin, exchange))
                elapsed_time = datetime.datetime.now() - dateutil.parser.parse(last_updated_time)
            nperiods_ago = 0
            if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex':
                nperiods_ago = elapsed_time/(60*60*24)
            elif period == '1hour' or period == '1hourbtcbitfinex':
                 nperiods_ago = elapsed_time/(60*60)
            """
            nperiods_ago = elapsed_time / datetime.timedelta(days=1 if period == '1day' or period == '1daycryptopia' or period =='1daybtcbitfinex' else 0,
                                                             hours=1 if period == '1hour' or period == '1hourbtcbitfinex' else 0,
                                                             minutes=1 if period == '1min' else 0,
                                                             seconds=1)
            """
            nperiods_ago = np.floor(nperiods_ago)
            #print(nperiods_ago)
            logging.info(PRINT_MSG.format(exchange, last_updated_time, elapsed_time, nperiods_ago))
            #if coin == 'BTC':
              #  nperiods_ago = nperiods_ago-1
            if nperiods_ago > 0:
                
                logging.info("Updating data for {coin}-{exchange} from {last_updated_time}".format(
                    coin=coin, exchange=exchange, last_updated_time=last_updated_time)
                )
                #sys.exit("Testing")
                new_data_coin_period = fetch_data_api(
                    coin=coin,
                    nperiods=nperiods_ago,
                    period=period,
                    exchange_name=exchange
                )
                #print(new_data_coin_period.shape)
                if new_data_coin_period is None:
                    print(coin,exchange," Info Not available from API",str(j))
                    j =j+1
                    continue
                new_data_coin_period['coin'] = coin
                new_data_coin_period['exchange'] = exchange
                new_data_coin_period = new_data_coin_period.reset_index()
                new_data_coin_period['time'] =  pd.to_datetime(new_data_coin_period.unix_timestamp,unit="s",utc=True)
                i = i + 1
                #print(coin,exchange,i)
                lst_new_data.append(new_data_coin_period)
    
    #print("CSV column order "+str(csv_column_order))
    if lst_new_data:
        df_new_data = pd.concat(lst_new_data)
        df_new_data = df_new_data.reset_index()
        curr_columns = df_new_data.columns.tolist()
        #print("Current columns "+str(curr_columns))
        df_coin_period = df_coin_period.reset_index()
        csv_column_order = df_coin_period.columns.tolist()
        column_order = [col for col in csv_column_order if col in curr_columns]
        #print("New Column Order "+str(column_order))
        df_new_data = df_new_data.reindex(columns=column_order)
        df_coin_period=df_coin_period.append(df_new_data)
        #df_new_data.to_csv(csv_filename, mode='a', header=False,index=False,date_format="%d-%m-%Y %H:%M:%S")
        df_coin_period = df_coin_period[final_csv_column_order]
        df_coin_period.drop_duplicates(subset=['coin','exchange','unix_timestamp'],inplace=True)
        df_coin_period.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
        #df_new_data.to_csv('Put it into CSV.csv')
    print("Done")    

In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']

update_csv_to_latest('1day',datetimeStringformat_to_csv,columns_order)

2018-07-16 11:10:54,833 - INFO - Exchange        Last Updated Time            Elapsed Time nPeriodsAgo
2018-07-16 11:10:54,851 - INFO - ---------- For coin - XWC
2018-07-16 11:10:54,853 - INFO - Bittrex         1531612800                         106854        1.0
2018-07-16 11:10:54,854 - INFO - Updating data for XWC-Bittrex from 1531612800


Updated Dataframe


2018-07-16 11:10:56,152 - INFO - ---------- For coin - XDN
2018-07-16 11:10:56,153 - INFO - Bittrex         1531612800                         106856        1.0
2018-07-16 11:10:56,154 - INFO - Updating data for XDN-Bittrex from 1531612800
2018-07-16 11:10:57,431 - INFO - ---------- For coin - EMC2
2018-07-16 11:10:57,433 - INFO - Bittrex         1531612800                         106857        1.0
2018-07-16 11:10:57,434 - INFO - Updating data for EMC2-Bittrex from 1531612800
2018-07-16 11:10:58,688 - INFO - ---------- For coin - THC
2018-07-16 11:10:58,690 - INFO - Bittrex         1531612800                         106858        1.0
2018-07-16 11:10:58,691 - INFO - Updating data for THC-Bittrex from 1531612800
2018-07-16 11:10:59,952 - INFO - ---------- For coin - VTR
2018-07-16 11:10:59,953 - INFO - Bittrex         1531612800                         106859        1.0
2018-07-16 11:10:59,954 - INFO - Updating data for VTR-Bittrex from 1531612800
2018-07-16 11:11:01,246 - INFO - -----

BSD Bittrex  Info Not available from API 0


2018-07-16 11:13:06,231 - INFO - Bittrex         1531612800                         106986        1.0
2018-07-16 11:13:06,232 - INFO - Updating data for XVG-Bittrex from 1531612800
2018-07-16 11:13:07,525 - INFO - ---------- For coin - MEME
2018-07-16 11:13:07,526 - INFO - Bittrex         1531612800                         106987        1.0
2018-07-16 11:13:07,527 - INFO - Updating data for MEME-Bittrex from 1531612800
2018-07-16 11:13:08,790 - INFO - ---------- For coin - GAME
2018-07-16 11:13:08,791 - INFO - Bittrex         1531612800                         106988        1.0
2018-07-16 11:13:08,792 - INFO - Updating data for GAME-Bittrex from 1531612800
2018-07-16 11:13:10,106 - INFO - ---------- For coin - STEEM
2018-07-16 11:13:10,108 - INFO - Binance         1531612800                         106990        1.0
2018-07-16 11:13:10,109 - INFO - Updating data for STEEM-Binance from 1531612800
2018-07-16 11:13:11,404 - INFO - Bittrex         1531612800                         106991 

SBD Bittrex  Info Not available from API 1


2018-07-16 11:13:24,465 - INFO - ---------- For coin - ETC
2018-07-16 11:13:24,466 - INFO - Binance         1531612800                         107004        1.0
2018-07-16 11:13:24,467 - INFO - Updating data for ETC-Binance from 1531612800
2018-07-16 11:13:25,707 - INFO - Bittrex         1531612800                         107005        1.0
2018-07-16 11:13:25,708 - INFO - Updating data for ETC-Bittrex from 1531612800
2018-07-16 11:13:26,989 - INFO - Kucoin          1531612800                         107006        1.0
2018-07-16 11:13:26,990 - INFO - Updating data for ETC-Kucoin from 1531612800
2018-07-16 11:13:28,322 - INFO - ---------- For coin - STRAT
2018-07-16 11:13:28,323 - INFO - Binance         1531612800                         107008        1.0
2018-07-16 11:13:28,324 - INFO - Updating data for STRAT-Binance from 1531612800
2018-07-16 11:13:29,659 - INFO - Bittrex         1531612800                         107009        1.0
2018-07-16 11:13:29,660 - INFO - Updating data for ST

EOS IDEX  Info Not available from API 2


2018-07-16 11:16:16,283 - INFO - ---------- For coin - MDA
2018-07-16 11:16:16,285 - INFO - Binance         1531612800                         107176        1.0
2018-07-16 11:16:16,286 - INFO - Updating data for MDA-Binance from 1531612800
2018-07-16 11:16:17,585 - INFO - ---------- For coin - MTL
2018-07-16 11:16:17,587 - INFO - Binance         1531612800                         107177        1.0
2018-07-16 11:16:17,588 - INFO - Updating data for MTL-Binance from 1531612800
2018-07-16 11:16:18,914 - INFO - IDEX            1531612800                         107178        1.0
2018-07-16 11:16:18,915 - INFO - Updating data for MTL-IDEX from 1531612800
2018-07-16 11:16:20,415 - INFO - ---------- For coin - SALT
2018-07-16 11:16:20,417 - INFO - Binance         1531612800                         107180        1.0
2018-07-16 11:16:20,418 - INFO - Updating data for SALT-Binance from 1531612800
2018-07-16 11:16:21,641 - INFO - Bittrex         1531612800                         107181        1.

In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'BBANDS_BANDWIDTH_PERCENT','MACD','MACD_SIGNAL','MACD_TEST','MONEY_FLOW_INDEX','ON_BALANCE_VOLUME','ON_BALANCE_VOLUME_TEST','RSI','RSI_OVER_BOUGHT',
                 'RSI_OVER_SOLD','SMA_FAST','SMA_SLOW','SMA_TEST','STOCHRSI_D','STOCHRSI_K','STOCH_OVER_BOUGHT','STOCH_OVER_SOLD','STOCH_PERCENT_D','STOCH_PERCENT_D_MONEY_FLOW_INDEX',
                 'STOCH_PERCENT_K','STOCH_PERCENT_K_MONEY_FLOW_INDEX']
update_csv_to_latest('1dayCryptopia',datetimeStringformat_to_csv,columns_order)

In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
update_csv_to_latest('1daybtcbitfinex',datetimeStringformat_to_csv,columns_order)

In [ ]:
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
update_csv_to_latest('1hourbtcbitfinex',datetimeStringformat_to_csv,columns_order)

In [ ]:
indicator_list_btc = ['unix_timestamp','UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']

In [ ]:
def update_indicator_BTC(csv_filename,periods,timeframe,datetimeformat_string):
    """ Update the given csv_file with new column values for corr rows """
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True)
    #df_csv.time = df_csv.time.apply(lambda t: datetime.datetime.strptime(t, '%d-%m-%Y %H:%M:%S'))
    
    for indicator in indicator_list_btc:
        if indicator not in df_csv.columns and indicator not in df_csv.index:
            df_csv[indicator] = np.nan
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv = df_csv.set_index(['coin', 'exchange','unix_timestamp'])
    data = list(df_csv.index.get_level_values(0).unique())
    i=0
    j=0
    for coin_name in data:
        coin_df = df_csv[df_csv.index.get_level_values(0)==coin_name] 
        #print(coin_df)
        coin_df = coin_df.reset_index()
        coin_df = coin_df.sort_values(by=['exchange','unix_timestamp']).set_index(['coin', 'exchange','unix_timestamp'])
        #print(coin_df)
        df_groupby = coin_df.groupby(['exchange'], group_keys=False)
        for key, item in df_groupby:
            req_data = df_groupby.get_group(key)
            req_data2 = req_data.iloc[-periods:]

            start_date = req_data2.index.get_level_values(2)[0]
            end_date = req_data2.index.get_level_values(2)[req_data2.shape[0]-1]
            req_data2 = req_data[(req_data.index.get_level_values(2) >= start_date) & (req_data.index.get_level_values(2) <= end_date)]
            #print(req_data2)
            np_volumeto = np.array(req_data2.volumeto.values,dtype='f8')
            if len(np_volumeto)<20:
                j = j+1
                print(coin_name,j," Not Updated")
                continue
            req_data2['UPPER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.upper_bollinger_band(req_data2.close.values,20)
            req_data2['MIDDLE_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.middle_bollinger_band(req_data2.close.values,20)
            req_data2['LOWER_BOLLINGER_BAND_VALUE'] = pyti.bollinger_bands.lower_bollinger_band(req_data2.close.values,20)
            df_csv.update(req_data2)
            i = i+1
            print(coin_name,i)
            #print(df_csv.query('coin == @coin_name').tail(1))
            #sys.exit("Testing")
    df_csv.to_csv(csv_filename,date_format=datetimeformat_string)
    print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_day_full_1day.csv',250,'1day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_day_full_3days.csv',250,'3day',datetimeStringformat_to_csv)
print("Done")  

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_day_full_7days.csv',250,'7day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_day_full_14days.csv',250,'14day',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_hour_full_1hour.csv',250,'1hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_hour_full_4hours.csv',250,'4hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_hour_full_6hours.csv',250,'6hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
update_indicator_BTC('D:\crypto_analysis-master-master\crypto_analysis-master-master\data\BTC_Bitfinex_hour_full_12hours.csv',250,'12hour',datetimeStringformat_to_csv)
print("Done")

In [ ]:
scheduler = BackgroundScheduler()
scheduler.start()
columns_order = ['coin','exchange','unix_timestamp','time','open','high','low','close','volumefrom','volumeto',
                 'UPPER_BOLLINGER_BAND_VALUE','MIDDLE_BOLLINGER_BAND_VALUE','LOWER_BOLLINGER_BAND_VALUE']
scheduler.add_job(update_csv_to_latest, "cron", ['1daybtcbitfinex',datetimeStringformat_to_csv,columns_order], minute='*/1')


In [ ]:
scheduler.shutdown(wait=False)

In [ ]:
def changeCSVDateTimeFormat(csv_filename):
    df_csv = pd.read_csv(csv_filename, index_col=None,dayfirst=True,infer_datetime_format =True)
    #df_csv.unix_timestamp =  df_csv.time.apply(lambda t: time.mktime(datetime.datetime.strptime(str(t) ,'%d-%m-%Y %H:%M').timetuple()))
    df_csv['time'] = pd.to_datetime(df_csv.unix_timestamp,unit="s",utc=True)
    df_csv.set_index(['coin', 'exchange','unix_timestamp']).to_csv(csv_filename,date_format=datetimeStringformat_to_csv)
    print("Done")

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('all_coins_day_full_1day_Cryptopia.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_day_full_1day.csv')

In [ ]:
changeCSVDateTimeFormat('BTC_Bitfinex_hour_full_1hour.csv')

In [ ]:
print( type(datetime.datetime.now()))